In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, output_size, pad_idx, hidden_size, dropout):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        self.lstm = nn.LSTM(embedding_size, hidden_size
                           , bidirectional=True, num_layers=2)
        self.linear = nn.Linear(hidden_size * 2, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        embedded = self.embed(text) # [seg_len, batch_size, embedding_size]
        embedded = self.dropout(embedded)
        output, (hidden, cell) = self.lstm(embedded)
        # hidden: 2 * batch_size * hidden_size
        hidden = torch.cat([hidden[-1], hidden[-2]], dim=1)
        hidden = self.dropout(hidden.squeeze())
        return self.linear(hidden)

In [3]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_SIZE = 100
OUTPUT_SIZE = 1
PAD_IDX = TEXT.vocab.stoi(TEXT.pad_token)
HIDDEN_SIZE = 100
DROPOUT = 0.5

model = RNNModel(vocab_size=VOCAB_SIZE, embedding_size=EMBEDDING_SIZE
                 , output_size=OUTPUT_SIZE, pad_idx=PAD_IDX, 
                 hidden_size=HIDDEN_SIZE, dropout=DROPOUT)

pretrained_embedding = TEXT.vocab.vectors
mdoel.embed.weight.data.copy_(pretrained_embedding)


UNK_IDX = TEXT.vocab.stoi(TEXT.unk_token)
mdoel.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
mdoel.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

NameError: name 'TEXT' is not defined

In [2]:
import torch
optimizer = torch.optim.Adam(model.parameters())
crit = nn.BCEWithLogitsLoss()

model = model.to(device)
crit = model.to(crit)



def binary_accuracy(preds, y):
    rounted_preds = torch.round(torch.sigmoid(preds))
    correct = (rounted_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.train()
    for batch in iterator:
        preds = model(batch.text).squeeze() #[squeeze]
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * len(batch_label)
        epoch_acc += acc.item() * len(batch_label)
        
        total_len += len(batch.label)
    return epoch_loss/len(iterator)

def evaluate(model, iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.eval()
    for batch in iterator:
        preds = model(batch.text).squeeze()
        loss = crit(preds, batch.label)
        acc = binary_accuracy(preds, batch.label)        
        
        epoch_loss += loss.item() * len(batch_label)
        epoch_acc += acc.item() * len(batch_label)
        
        total_len += len(batch.label)
    model.train()
    return epoch_loss/len(iterator)

N_EPOCHS = 10
best_valid_acc = 0.
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, trian_iterator, optimizer, crit)
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'lstm-model.pth')
    print("Epoch", epoch, "Train Loss", train_loss, "Train Acc", train_acc)
    print("Epoch", epoch, "Valid Loss", valid_loss, "Valid Acc", valid_acc)

NameError: name 'torch' is not defined

In [ ]:
# CNN, ngram feature detecter